<a href="https://colab.research.google.com/github/hamidriasat/Natural-Language-Processing/blob/master/Hotel_Review_Prediction/Hotel_Reviews_XLnet_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hotel Reviews Project 
Model deployment code file

First get pretrained model files


In [ ]:

!gdown --id 1-5SmI5cKTNfj2TelCTGg_jmyOsNsE9rP
!gdown --id 1-5_WkT8QoFOLyZByWAi9FwD2laN-_AJc
!gdown --id 1-D7S6hidsetfSUX2QUMH7Mn1-B5pivH9
!gdown --id 1-ArHf0f08u_WI42TPi4Yt_8rwux7CQne
!gdown --id 1-IiCA5hKj603PsYCulYO--XHRV9W7jzD


Downloading...
From: https://drive.google.com/uc?id=1-5SmI5cKTNfj2TelCTGg_jmyOsNsE9rP
To: /content/config.json
100% 1.08k/1.08k [00:00<00:00, 2.14MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-5_WkT8QoFOLyZByWAi9FwD2laN-_AJc
To: /content/pytorch_model.bin
469MB [00:04, 113MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-D7S6hidsetfSUX2QUMH7Mn1-B5pivH9
To: /content/special_tokens_map.json
100% 202/202 [00:00<00:00, 364kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-ArHf0f08u_WI42TPi4Yt_8rwux7CQne
To: /content/spiece.model
100% 798k/798k [00:00<00:00, 53.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-IiCA5hKj603PsYCulYO--XHRV9W7jzD
To: /content/tokenizer_config.json
100% 238/238 [00:00<00:00, 413kB/s]


# install pytorch-transformers libarary

and make all the necessary imports

In [ ]:
!pip install transformers


     |████████████████████████████████| 1.3MB 9.0MB/s 
     |████████████████████████████████| 890kB 31.4MB/s 
     |████████████████████████████████| 1.1MB 55.4MB/s 
     |████████████████████████████████| 2.9MB 55.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=23bee8199835be2bed55bae0cc8ad13ab1d86897b8ea78e8ed1ba3b7e8cb6c75
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
from xlrd import open_workbook
import csv


from transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define model parameters

In [ ]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES["xlnet"]

# Load model in testing mode

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
path_to_modle= "/content/"
model = model_class.from_pretrained( path_to_modle)
tokenizer = tokenizer_class.from_pretrained( path_to_modle)
# Copy the model to the GPU.
model.to(device)
model.eval() #in evaluation mode


/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `mem_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e

# Get Testing dataset
For reference we are using excel file

You can change and load data according to your requirements


In [ ]:
!gdown --id 1UkYwj-PZ4xPs9D0lI30MW7ASIEfzLWUt

Downloading...
From: https://drive.google.com/uc?id=1UkYwj-PZ4xPs9D0lI30MW7ASIEfzLWUt
To: /content/deploymentInput.xlsx
100% 9.21k/9.21k [00:00<00:00, 13.1MB/s]


# Data Input
you can give one or multiple data points at the same time for classification

Output is sequential, meaning order of output will be same as order of input data points

# Load Input Data 
Unlike training dat here we are reading data from Excel file.
where each row represent one review.

ouput  is a list containing the text of input reviews

In [ ]:
# Set the maximum sequence length. 
MAX_LEN = 200

wb = open_workbook('deploymentInput.xlsx')
for sheet in wb.sheets():
    number_of_rows = sheet.nrows
    number_of_columns = sheet.ncols
data_X= []
input_data = []   
for row in range(0,number_of_rows):
    value  = (sheet.cell(row,0).value)
    input_data.append(value)
    templength= len(value.split() )
    if templength < MAX_LEN:
      data_X.append(value)
    else:
      temp= ""
      convert= value.split()
      for j in range(0, MAX_LEN):
        temp= temp+ " "+convert[j]
      data_X.append(temp)
    
print(len(data_X))

3


# Preprocessing Data

In [ ]:
# Create sentence and label lists
sentences = data_X

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# Use the XLNet tokenizer to convert the tokens to their index numbers in the XLNet vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Convert all of our data into torch tensors, the required datatype for our model

input_ids = torch.tensor(input_ids).to(device)
attention_masks = torch.tensor(attention_masks).to(device)


In [ ]:
# Set the batch size.  
batch_size = 16  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size
                                   ,shuffle= False)

# Prediction


In [ ]:

# Tracking variables 
predictions = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  logits= np.argmax(logits, axis=1).flatten()
  
  # Store predictions and true labels
  predictions += list(logits)
print(' DONE.')

 DONE.


# Result


In [ ]:
print("Review #", "==>","Predicted Rating")

for i in range(len(predictions)):
  print("Review",(i+1), "==>",predictions[i])


Review # ==> Predicted Rating
Review 1 ==> 4
Review 2 ==> 5
Review 3 ==> 4


In [ ]:
with open('Result.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # add header
    writer.writerow(["Review Text", "Rating"])

    # write data
    for i in range(len(predictions)):  
      writer.writerow([input_data[i], predictions[i]])

print("Results.csv file created Successfully")

Results.csv file created Successfully
